###  Count Connectivity Components

Given the following grid:
<pre>  
+--+--+--+
|     |  |
+  +--+--+
|     |  |
+--+--+--+
</pre>  
we have the connectivity components described by [(4, 1), (1, 2)].

Your job is to define a function components(grid) that takes as argument a string representing a grid like in the above pictures and returns a list describing the size and number of the connectivity components. The list should be sorted in descending order by the size of the connectivity components. The grid may have any number of rows and columns.

Note: The grid is always rectangular and will have all its outer walls. Only inner walls may be missing. The + are considered bearing pillars, and are always present.

In [126]:
### Uses Connected-component labeling 2-pass algorithm (and disjoint sets)

def set_root(i, root_value, roots):
    """ Set root_value for every value in set """
    while roots[i] > root_value:
        ni = roots[i]
        roots[i] = root_value
        i = ni   
        
def find_min_root(i, roots):
    while roots[i] < i: i = roots[i]
    return i

def components(grid):
    if not grid: return []
    lines = grid.split("\n")
    y_size = len(lines) // 2
    if y_size <= 0: return []
    x_size = lines[0].count('+') - 1
    if x_size < 1: return []
    mat = [ [0] * x_size for i in range(0, y_size) ]
    # iterate over cells and populate matrix with initial values to count number of areas
    i = 0
    y = 0
    counter = {} #counter for area-size
    roots = {}
    for l in range(1, len(lines), 2):
        x = 0
        for c in range(1, len(lines[l]), 3): 
            v = 0
            if lines[l-1][c] == '-' and lines[l][c-1] == '|': # if closed from up and left
                i += 1 #increase area counter
                v = i
                roots[v] = v 
                counter[v] = 0 # initialize counter for i-size
            elif lines[l-1][c] == '-':
                v = mat[y][x-1]
            elif lines[l][c-1] == '|':
                v = mat[y-1][x]
            else:
                v = min(mat[y][x-1], mat[y-1][x])
                m = max(mat[y][x-1], mat[y-1][x])
                if v < m:
                    rv = find_min_root(v, roots)
                    rm = find_min_root(m, roots)
                    if rv < rm:
                        set_root(m, rv, roots)
                    elif rm < rv:
                        set_root(v, rm, roots)
            mat[y][x] = v              
            x += 1
        y += 1 
    for l in mat: print(l)
    # 2 pass - set root value to cells
    for l in mat:
        for c in range(0, x_size):
            v = find_min_root(l[c], roots)
            l[c] = v
            counter[v] = counter[v] + 1   
    print("----")
    for l in mat: print(l)
    # reduce area sets to size based sets
    size_dict = {}
    for i, size in counter.items():
        if size > 0:
            size_dict[size] = size_dict.get(size,0) + 1

    return  sorted([ (size, count) for size, count in size_dict.items() ], key = lambda x: x[0], reverse = True)

In [127]:
grid = "+--+--+--+\n|     |  |\n+  +--+--+\n|     |  |\n+--+--+--+"
print(components(grid))

[1, 1, 2]
[1, 1, 3]
----
[1, 1, 2]
[1, 1, 3]
[(4, 1), (1, 2)]


In [128]:
components('''\
+--+--+--+
|  |  |  |
+--+--+--+
|  |  |  |
+--+--+--+''')
#, [(1, 6)])



[1, 2, 3]
[4, 5, 6]
----
[1, 2, 3]
[4, 5, 6]


[(1, 6)]

In [129]:
components('''\
+--+--+--+
|  |     |
+  +  +--+
|  |  |  |
+--+--+--+''')
#, [(3, 1), (2, 1), (1, 1)])

[1, 2, 2]
[1, 2, 3]
----
[1, 2, 2]
[1, 2, 3]


[(3, 1), (2, 1), (1, 1)]

In [130]:
components('''\
+--+--+--+
|  |     |
+  +  +--+
|        |
+--+--+--+''')
#, [(6, 1)])


[1, 2, 2]
[1, 1, 1]
----
[1, 1, 1]
[1, 1, 1]


[(6, 1)]

In [131]:
components('''\
+--+--+--+
|        |
+  +  +  +
|        |
+--+--+--+''')
#, [(6, 1)])

[1, 1, 1]
[1, 1, 1]
----
[1, 1, 1]
[1, 1, 1]


[(6, 1)]

In [132]:
components('''\
+--+--+--+--+--+--+--+--+--+--+
|  |     |  |     |  |     |  |
+--+--+--+  +  +  +--+--+--+--+
|        |     |  |     |  |  |
+  +--+--+--+--+--+  +--+  +  +
|     |  |  |  |  |     |  |  |
+--+--+  +  +--+--+--+  +  +--+
|        |  |  |  |  |  |     |
+  +--+  +  +  +--+--+--+--+--+
|        |  |     |  |        |
+  +--+--+  +--+  +  +  +--+--+
|  |  |           |  |        |
+--+  +--+--+--+  +  +--+--+  +
|  |  |  |  |  |  |  |  |  |  |
+  +--+--+--+  +--+--+--+--+--+
|  |     |     |  |  |     |  |
+--+  +--+--+  +  +--+--+  +  +
|        |     |     |        |
+--+--+  +  +  +--+  +  +--+  +
|     |  |     |  |        |  |
+--+--+--+--+--+--+--+--+--+--+''')
#[(11, 1), (8, 1), (7, 3), (6, 3), (5, 2), (4, 1), (3, 1), (2, 6), (1, 13)] 
#[(13, 1), (11, 1), (8, 1), (7, 2), (6, 2), (5, 2), (4, 1), (3, 1), (2, 6), (1, 13)]

[1, 2, 2, 3, 4, 4, 5, 6, 6, 7]
[8, 8, 8, 3, 3, 4, 9, 9, 10, 11]
[8, 8, 12, 13, 14, 15, 9, 9, 10, 11]
[16, 16, 12, 13, 17, 18, 19, 9, 10, 10]
[16, 16, 12, 13, 17, 17, 20, 21, 21, 21]
[16, 22, 23, 13, 13, 13, 20, 21, 21, 21]
[24, 22, 25, 26, 27, 13, 20, 28, 29, 21]
[24, 30, 30, 31, 27, 32, 33, 34, 34, 35]
[36, 30, 30, 37, 27, 32, 32, 38, 34, 34]
[39, 39, 30, 37, 27, 40, 32, 32, 32, 34]
----
[1, 2, 2, 3, 3, 3, 5, 6, 6, 7]
[8, 8, 8, 3, 3, 3, 9, 9, 10, 11]
[8, 8, 12, 13, 14, 15, 9, 9, 10, 11]
[12, 12, 12, 13, 13, 18, 19, 9, 10, 10]
[12, 12, 12, 13, 13, 13, 20, 21, 21, 21]
[12, 22, 13, 13, 13, 13, 20, 21, 21, 21]
[24, 22, 25, 26, 27, 13, 20, 28, 29, 21]
[24, 30, 30, 27, 27, 32, 33, 32, 32, 32]
[30, 30, 30, 27, 27, 32, 32, 32, 32, 32]
[39, 39, 30, 27, 27, 40, 32, 32, 32, 32]


[(13, 1),
 (11, 1),
 (8, 1),
 (7, 2),
 (6, 2),
 (5, 2),
 (4, 1),
 (3, 1),
 (2, 6),
 (1, 13)]

In [133]:
components('''\
+--+--+--+--+--+--+--+--+--+--+
|  |     |     |  |  |  |     |
+--+--+--+--+--+--+--+--+--+  +
|  |  |     |  |  |     |  |  |
+--+--+--+  +  +  +--+--+--+--+
|  |        |     |  |  |  |  |
+--+--+--+--+  +--+  +  +  +--+
|  |  |  |     |  |  |  |  |  |
+  +--+--+--+  +  +  +  +  +--+
|     |     |  |     |  |  |  |
+--+  +  +  +--+  +--+--+--+  +
|     |        |  |           |
+--+--+--+  +--+--+--+--+--+--+
|  |        |  |           |  |
+  +  +--+  +  +--+--+  +  +--+
|        |     |  |  |  |     |
+--+--+  +  +  +--+  +--+--+--+
|  |  |  |  |  |  |  |  |  |  |
+--+--+--+--+--+  +  +--+  +--+
|  |     |        |  |     |  |
+--+--+--+--+--+--+--+--+--+--+''')
#[(13, 1), (7, 2), (6, 1), (5, 4), (4, 1), (3, 5), (2, 4), (1, 20)] 
#should equal 
#[(18, 1), (7, 2), (6, 1), (5, 3), (4, 1), (3, 5), (2, 4), (1, 20)]

[1, 2, 2, 3, 3, 4, 5, 6, 7, 7]
[8, 9, 10, 10, 11, 12, 13, 13, 14, 7]
[15, 16, 16, 10, 11, 11, 17, 18, 19, 20]
[21, 22, 23, 24, 11, 25, 17, 18, 19, 26]
[21, 21, 27, 27, 11, 25, 17, 18, 19, 28]
[29, 21, 27, 27, 27, 25, 30, 30, 30, 28]
[31, 32, 32, 27, 33, 34, 34, 34, 34, 35]
[31, 31, 31, 27, 27, 36, 37, 34, 34, 34]
[38, 39, 31, 27, 27, 40, 37, 41, 42, 43]
[44, 45, 45, 46, 46, 40, 37, 47, 42, 48]
----
[1, 2, 2, 3, 3, 4, 5, 6, 7, 7]
[8, 9, 10, 10, 11, 11, 13, 13, 14, 7]
[15, 10, 10, 10, 11, 11, 17, 18, 19, 20]
[21, 22, 23, 11, 11, 17, 17, 18, 19, 26]
[21, 21, 27, 27, 11, 17, 17, 18, 19, 28]
[21, 21, 27, 27, 27, 17, 28, 28, 28, 28]
[27, 27, 27, 27, 27, 34, 34, 34, 34, 35]
[27, 27, 27, 27, 27, 36, 37, 34, 34, 34]
[38, 39, 27, 27, 27, 40, 37, 41, 42, 43]
[44, 45, 45, 40, 40, 40, 37, 42, 42, 48]


[(18, 1), (7, 2), (6, 1), (5, 3), (4, 1), (3, 5), (2, 4), (1, 20)]

In [134]:
components('''\
+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+
|  |        |  |           |           |        |     |     |
+--+--+--+--+  +  +--+--+--+  +  +--+--+--+  +  +--+--+--+--+
|  |  |  |           |        |  |  |  |  |  |        |  |  |
+--+  +--+  +  +  +--+  +--+  +--+--+  +  +  +--+--+--+--+--+
|  |  |  |           |  |  |     |     |        |  |  |     |
+  +--+--+--+--+--+  +--+--+--+  +--+--+--+  +--+--+--+  +  +
|  |     |        |  |              |  |     |     |  |     |
+  +--+  +--+--+  +  +  +  +--+  +  +  +--+--+--+--+--+  +  +
|     |     |  |  |        |  |     |        |        |  |  |
+  +--+--+--+  +--+  +  +  +--+--+  +--+--+--+  +--+  +--+  +
|  |     |     |           |  |     |  |     |  |           |
+  +--+--+--+--+  +  +--+--+--+--+--+  +  +--+  +--+  +--+  +
|     |  |     |  |     |     |  |  |  |     |  |     |     |
+--+--+--+  +--+--+--+  +--+  +  +  +--+  +--+--+  +--+  +  +
|  |     |           |           |     |     |  |     |     |
+  +--+--+--+  +  +--+  +--+--+  +  +  +--+--+  +--+--+--+--+
|  |     |  |     |        |  |  |     |              |  |  |
+  +  +  +--+  +--+  +  +--+  +--+--+  +--+  +  +  +  +  +--+
|           |  |  |     |     |  |     |  |  |     |        |
+--+  +--+  +--+--+--+  +  +--+  +  +  +  +--+--+  +  +  +  +
|  |  |  |     |  |  |  |  |        |           |           |
+--+--+--+--+--+  +  +  +--+  +  +--+  +  +--+--+  +--+--+--+
|  |  |  |              |     |  |     |  |  |  |     |  |  |
+  +--+  +--+  +--+--+  +  +  +--+  +  +--+--+  +  +--+--+  +
|  |  |  |                    |  |  |  |  |  |     |        |
+--+  +--+  +--+  +--+--+  +--+  +  +--+--+  +--+  +  +--+  +
|  |     |           |  |  |  |        |  |  |  |  |        |
+  +  +--+--+  +--+  +  +--+  +--+--+--+--+  +--+  +  +  +--+
|        |  |        |  |  |  |  |     |  |  |  |           |
+  +--+--+--+  +  +--+--+  +  +  +  +--+--+  +  +--+  +  +  +
|  |     |  |     |           |     |  |  |     |           |
+  +  +  +  +--+  +  +  +  +--+--+  +  +--+  +--+--+  +--+--+
|           |     |  |           |  |  |        |     |     |
+  +  +--+--+  +  +--+--+--+--+  +--+  +--+  +--+  +--+--+--+
|                 |     |        |  |  |        |     |     |
+  +--+--+--+--+  +--+  +  +--+  +  +--+--+  +--+  +  +--+  +
|           |  |                    |  |  |     |        |  |
+--+--+--+  +  +--+  +--+--+  +--+  +--+--+  +--+--+--+  +--+
|  |  |  |        |  |  |  |                 |  |  |  |  |  |
+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+''')
#[(182, 1), (46, 1), (23, 1), (13, 1), (11, 1), (9, 1), (7, 1), (6, 2), (5, 1), (4, 4), (3, 5), (2, 11), (1, 39)] 
#should equal 
#[(187, 1), (46, 1), (23, 1), (13, 1), (11, 1), (9, 1), (7, 1), (6, 2), (4, 4), (3, 5), (2, 11), (1, 39)]

[1, 2, 2, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8]
[9, 10, 11, 12, 3, 3, 3, 13, 13, 5, 5, 14, 15, 16, 6, 6, 6, 6, 17, 18]
[19, 10, 20, 12, 3, 3, 3, 13, 21, 5, 5, 22, 15, 16, 6, 6, 23, 24, 25, 25]
[19, 26, 26, 27, 27, 27, 3, 28, 28, 28, 5, 5, 29, 30, 6, 31, 31, 32, 25, 25]
[19, 19, 26, 26, 33, 27, 3, 3, 3, 34, 5, 5, 29, 29, 29, 35, 35, 35, 25, 25]
[19, 36, 36, 37, 33, 38, 3, 3, 3, 39, 40, 5, 41, 42, 42, 35, 43, 35, 35, 25]
[19, 19, 44, 45, 45, 38, 3, 3, 46, 46, 47, 48, 41, 42, 42, 35, 49, 35, 50, 25]
[51, 52, 52, 45, 45, 45, 45, 3, 3, 3, 3, 48, 48, 42, 42, 53, 49, 49, 50, 25]
[51, 54, 54, 55, 45, 45, 56, 3, 3, 57, 3, 48, 48, 58, 58, 53, 53, 53, 59, 60]
[51, 51, 51, 51, 45, 61, 56, 3, 62, 57, 63, 64, 48, 65, 58, 53, 53, 53, 53, 53]
[66, 51, 67, 51, 51, 68, 69, 3, 62, 70, 63, 63, 48, 48, 48, 48, 53, 53, 53, 53]
[71, 72, 73, 74, 74, 68, 68, 3, 75, 70, 63, 76, 48, 48, 77, 78, 53, 53, 79, 80]
[71, 81, 73, 82, 74, 74, 74, 3, 3, 3, 83, 76, 48, 84, 85, 78, 53, 86, 86, 80]
[87, 81, 81,

[(187, 1),
 (46, 1),
 (23, 1),
 (13, 1),
 (11, 1),
 (9, 1),
 (7, 1),
 (6, 2),
 (4, 4),
 (3, 5),
 (2, 11),
 (1, 39)]